Preguntas
* Distinguir entre materiales y texturas
* Obtener la url desde el data
* Software compatible ¿lista?
* Tags ¿lista?

In [13]:
import random, requests, time, csv, re, os
from lxml import html
from urllib.parse import urljoin, urlparse

if not os.path.exists('data'):
    os.makedirs('data')

def updateSession(act):
    act = requests.Session()
    return act

def updateUserAgent():
    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
    ]
    headers={"User-Agent": user_agent_list[random.randint(0, len(user_agent_list)-1)]}
    return headers
def propiedades(text, patrones):
    dicc = {}
    for key, patron in patrones.items():
        match = re.search(patron, text)
        if match:
            dicc[key] = match.group(1)
    return dicc 
def get_important(tree, xpath_code):
    dicc = {}
    propierties = tree.xpath(xpath_code)
    for element in propierties:
        text = element.text_content()
        patrones = {
            'file_size': r'File Size: (.+)',
            'id': r'Model ID: ([#]\d+)',
            'polygons': r'Polygons: (\d+|N/A)',
            'vertex': r'Vertices: (\d+|N/A)',
            'is_rigged': r'Rigged: (\w{2,3})',
            'is_animated': r'Animated: (\w{2,3})',
            'license': r'License: (.+)',
            'quality': r'Quality: (.+)',
            'aviable_formats': r'Formats: ((\.\w+)(,\s*\.\w+)*|(\.))',
        }
        dicc.update(propiedades(text, patrones))
    return dicc

def get_software(tree, xpath_code):
    dicc = {}
    properties = tree.xpath(xpath_code)
    for element in properties:
        text = element.text_content()
        texto = " ".join(text.strip().split())#Poner el texto en una solo linea
        patrones = {
            'textures': r'Textures & Materials:((\.\w+)(,\s*\.\w+)*)',
            'software': r'Software for Import:(.*?)\.\.\.',
            'materials': r'\.\w+ for (.+?) Categories',
        }
        dicc.update(propiedades(texto, patrones))

    return dicc
def get_name(tree, xpathCode):
    dicc={}
    title = tree.xpath(xpathCode)

    Name= title[0].text_content()
    '''Name'''
    patronName = r'(.+) 3D Model'
    searchName = re.search(patronName,Name)
    if searchName:
        dicc['name'] = searchName.group(1)
    return dicc
def get_dscp(tree, xpath_code):
    dicc = {}
    Dscp = tree.xpath(xpath_code)
    if len(Dscp)>0:
        dicc['description']=Dscp[0].text_content()
    else:
        dicc['description']='Not Found'
    return dicc

def cat_tag(respuesta):
    dicc={}
    redirection_root_lxml=html.fromstring(respuesta.text)
    categories=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Categories:')]/parent::p/parent::div/a[not(contains(@rel,'tag'))]/text()")
    tags=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Tags: ')]/parent::p/following-sibling::a/text()")
    dicc={'category':categories,'tags':tags}
    return dicc

class Page:
    def __init__(self,headers):
        self.url=None
        self.next_page_url=None
        self.parsed_response=None
        self.headers=headers
        self.sess = requests.Session()

    def setUrl(self,url):
        self.url=url

    def getParsedResponse(self):
        response=self.sess.get(self.url,headers=self.headers)
        self.parsed_response=html.fromstring(response.text)

    def getItemsLinks(self):
        self.getParsedResponse()
        items=self.parsed_response.xpath("//div[contains(@id,'site-')]//div[@class='itemtitle']/a/@href")
        return items
    
    def setNextPageUrl(self):
        self.getParsedResponse()
        self.next_page_url=self.parsed_response.xpath("//a[@class='nextpostslink' and @rel='next' and @aria-label='Next Page']/@href")[0]
        
    def getMaxPags(self):
        self.getParsedResponse()
        max_pags=self.parsed_response.xpath("//div[@class='nav-links']/a[last()-1]/text()")[0]
        return max_pags
    
def downloadModel(urlModel, session):
    response = session.get(urlModel)
    tree = html.fromstring(response.content)

    enlace_pag_download = tree.xpath('/html/body/div[3]/article/div[3]/div[1]/div[3]/div[1]/div[2]/a/@href')

    if enlace_pag_download:
        enlace_pag_download = enlace_pag_download[0]
        response_download_page = requests.get(enlace_pag_download)
        tree_download_page = html.fromstring(response_download_page.content)
        enlace_descarga = tree_download_page.xpath('/html/body/div[3]/div[7]/div/div/div[6]/a/@href')
        
        if enlace_descarga:
            enlace_descarga = enlace_descarga[0]

            # Construir la URL completa
            url_base = 'https://open3dmodel.com'
            url_completa = urljoin(url_base, enlace_descarga)
            print(f"URL completa: {url_completa}")
            # Realizar la descarga y guardar el archivo
            time.sleep(random.uniform(1,3))
            response = session.get(url_completa)

            # Verificar si la descarga fue exitosa (código de respuesta 200)
            if response.status_code == 200:
                print('response url', response.url)
                formulario_xpath = '/html/body/div[1]/div[2]/form'
                root = html.fromstring(response.text)
                formulario = root.xpath(formulario_xpath)
                if formulario != []:
                    params = {}
                    # Obtiene y imprime los atributos de los elementos de entrada dentro del formulario
                    for input_element in formulario[0].xpath('.//input'):
                        nombre = input_element.get('name')
                        valor = input_element.get('value')
                        params[nombre] = valor
                        print(f"{nombre}: {valor}")
                        
                    response = session.get(response.url, params=params)
                    time.sleep(random.uniform(0,1))
                # Extraer información del encabezado Content-Disposition si está presente
                content_disposition = response.headers.get('Content-Disposition')
                
                if content_disposition and 'filename' in content_disposition:
                    # Obtener el nombre del archivo del encabezado Content-Disposition
                    filename = content_disposition.split('filename=')[1]
                    filename = filename.strip('"')
                    print('encabezado Content-Dispositiono', filename)
                else:
                    # Si no hay encabezado Content-Disposition, obtener el nombre del archivo de la URL
                    parsed_url = urlparse(url)
                    filename = parsed_url.path.split('/')[-1]
                    print('Si no hay encabezado', filename)

                filename = 'data/' + filename
                print(filename)
                # Guardar el contenido de la respuesta en un archivo local con el nombre extraído
                with open(filename, 'wb') as file:
                    file.write(response.content)
                print(f'Archivo descargado correctamente como {filename}')
                return filename
            else:
                print(f'Error al descargar el archivo. Código de estado: {response.status_code}')
                print(response.text)
                return None
        else:
            print("No se pudo encontrar el enlace de descarga en la página de descarga.")
            return None
    else:
        print("No se pudo encontrar el enlace a la página de descarga.")
        return None

Datos=['path',
       'file_size', 'id', 'name',
       'description','uri','aviable_formats',
       'textures','materials','polygons',
       'vertex','is_rigged', 'is_animated','license',
       'quality','software','category','tags'
       ]

'''
path: Ruta al archivo descargado (en el subdirectorio data).
file_size: Tamaño del archivo.
id: ID del modelo en la web.
name: Nombre del modelo.
description: Descripción.
uri: URI de la página del modelo (por ejemplo, https://open3dmodel.com/3d-models/star-cartoon-character_583501.html).
available_formats: Formatos disponibles.
textures_and_materials: Suma del número de texturas y materiales. Si no hay, 0.
polygons: Número de polígonos.
vertex: Número de vértices.
is_rigged: Rigged o no.
is_animated: Animado o no.
license: Licencia.
quality: Calidad.
software: Software compatible.
categories: Categorías.
tags: Tags.
'''


initial_url='https://open3dmodel.com/3d-models/' #page1
headers=updateUserAgent()
page=Page(headers)
i,m=0,0
max_pags=90
with open("Modelos_3D.csv", mode="w", newline='',encoding='utf-8') as archivo: 
    escritura = csv.writer(archivo)
    #Write de header of the file
    escritura.writerow(Datos)
    while i<max_pags:
        print(f"Numero de pagina: {i+1}")
        if i==0:
            url=initial_url
        page.setUrl(url)
        page.setNextPageUrl()
        models_links=page.getItemsLinks()
        print('Max: ',len(models_links))
        for link in models_links:
                #print(link)
                ori = page.sess
                response = ori.get(link,headers=headers)
                path = downloadModel(link, ori)
                if path != None:
                    pathdicc = {'path': str(path)}
                else:
                    pathdicc = {'path': 'not found'}
                tree = html.fromstring(response.content)
                properties = get_important(tree, '/html/body/div[3]/div/div[5]/ul/li')
                software=get_software(tree,'/html/body/div[3]/article/div[3]/div[1]/div[3]/div[2]/div[5]')
                description=get_dscp(tree,'/html/body/div[3]/article/div[3]/div[1]/div[3]/div[1]/div[1]/p')
                name=get_name(tree,'/html/body/div[3]/article/div[3]/div[1]/div[3]/h1')   
                link={'uri': response.url}
                print('filesize', Propordenadas['file_size'])
                propleft=cat_tag(response)
                Propiedades= pathdicc|properties|software|description|name|link|propleft #Only works in python versions > 3.9
                Propordenadas = {key: Propiedades[key] for key in Datos} 
                m+=1
                print(f'Modelo extraido no. {m}')
                escritura.writerow(list(Propordenadas.values()))
        url=page.next_page_url
        time.sleep(random.uniform(1,3))
        i=i+1


Numero de pagina: 1
Max:  42
URL completa: https://open3dmodel.com/?token_id=VlUyQjQ3TWp2WEZiMTBTRGVlMEVsQT09&key=MGdEdVRnM0hYSHE3WElxbVdSOHdFZz09
response url https://drive.usercontent.google.com/download?id=1H_q0ckfx98KJE15wEeEP12IdIiWiD4qY&export=download
encabezado Content-Dispositiono 9244_open3dmodel.zip
data/9244_open3dmodel.zip
Archivo descargado correctamente como data/9244_open3dmodel.zip
filesize 57.9 MB 
Modelo extraido no. 1
URL completa: https://open3dmodel.com/?token_id=NUt4L1REQkZmYW5CNnVSSWFLZzF1Zz09&key=dEZiM3ZuZXVCUm9Md25tMUNWTjI0dz09
response url https://drive.usercontent.google.com/download?id=143Ll34snjpkT2O4uEsGfechNjcRwTnBf&export=download
encabezado Content-Dispositiono 342_Open3DModel.com_Exclusive_06.zip
data/342_Open3DModel.com_Exclusive_06.zip
Archivo descargado correctamente como data/342_Open3DModel.com_Exclusive_06.zip
filesize 7.7 MB 
Modelo extraido no. 2
URL completa: https://open3dmodel.com/?token_id=MSt2NnNBNVdxUE5HZkhXZ1UrMEFPZz09&key=V0VVbkZkWWFmV

KeyboardInterrupt: 

In [2]:
import requests 
from lxml import html

url = 'https://open3dmodel.com/3d-models/3d-character-human-body-3dsmax-model_20474.html'
xpath_code = '//*[@id="gallery"]/div[1]/div[3]/div[1]/div[2]/a'
xpath='//*[@id="page"]/div[7]/div/div/div[6]/a'

# Realizar la primera solicitud GET
response = requests.get(url)
tree = html.fromstring(response.content)
element = tree.xpath(xpath_code)
href = element[0].get('href')

# Hacer otra petición GET al enlace obtenido
response = requests.get(href)
tree = html.fromstring(response.content)
element = tree.xpath(xpath)
download_link = element[0].get('href')

# Descargar el archivo directamente desde el enlace
file_name = download_link.split('/')[-1]
with open(file_name, 'wb') as f:
    f.write(requests.get(download_link).content)

print(f"Archivo '{file_name}' descargado con éxito.")



OSError: [Errno 22] Invalid argument: '?token_id=b0RMUlkwZ3BQTW5UOEh0TFp3VmFKQT09&key=d1orR1FNOHYwVE5uU2pMUHJmK3NXZz09'

In [11]:
import requests
from lxml import html
import time
import random

def updateSession(act):
    act = requests.Session()
    return act

def updateUserAgent():
    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
    ]
    headers={"User-Agent": user_agent_list[random.randint(0, len(user_agent_list)-1)]}
    return headers

class Page:
    def __init__(self,headers):
        self.url=None
        self.next_page_url=None
        self.parsed_response=None
        self.headers=headers
        self.sess = requests.Session()

    def setUrl(self,url):
        self.url=url

    def getParsedResponse(self):
        response=self.sess.get(self.url,headers=self.headers)
        self.parsed_response=html.fromstring(response.text)

    def getItemsLinks(self):
        self.getParsedResponse()
        items=self.parsed_response.xpath("//div[contains(@id,'site-')]//div[@class='itemtitle']/a/@href")
        return items
    
    def setNextPageUrl(self):
        self.getParsedResponse()
        self.next_page_url=self.parsed_response.xpath("//a[@class='nextpostslink' and @rel='next' and @aria-label='Next Page']/@href")[0]
        
    def getMaxPags(self):
        self.getParsedResponse()
        max_pags=self.parsed_response.xpath("//div[@class='nav-links']/a[last()-1]/text()")[0]
        return max_pags
        
initial_url='https://open3dmodel.com/3d-models/' #page1
page=Page(headers)
i, m=0, 0
max_pags = 1
csv_filename = 'output.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the header row
    header_row = ['path', 'file_size', 'id', 'name', 'description', 'uri', 'browse_file_formats', 'materials', 'polygons', 'vertex', 'is_rigged', 'is_animated', 'license', 'quality', 'software_for_import', 'categories', 'tags']
    csv_writer.writerow(header_row)
    while i < max_pags :
        print("new loop started")
        if i==0:
            url=initial_url
        page.setUrl(url)
        page.setNextPageUrl()
        models_links=page.getItemsLinks()
        max_pags = len(models_links)
        print('max', len(models_links))
        for link in models_links:
            ori = page.sess
            redirection_response=ori.get(link,headers=headers)
            redirection_root_lxml=html.fromstring(redirection_response.text)
            path = '' #path: Ruta al archivo descargado (en un subdirectorio data).
            file_size=redirection_root_lxml.xpath("//span[text()='File Size: ']/following-sibling::p/text()")
            id=redirection_root_lxml.xpath("//span[text()='Model ID: ']/following-sibling::p/text()")
            name=redirection_root_lxml.xpath("//h1[@class='entry-title']/text()")
            name = name[0].replace('\n', '')
            description=redirection_root_lxml.xpath("//div[@class='post-text-full']/div[@class='post-description entry-content']/p/text()")
            description=description[0].replace(',','|')
            uri=''
            browse_file_formats=redirection_root_lxml.xpath("//span[@class='notranslate']//a[starts-with(@href,'https://open3dmodel.com/')]/text()")
            materials=redirection_root_lxml.xpath("//span[text()='Textures & Materials:']/parent::p/following-sibling::p/text()")#caution!,this must be the number of materials
            materials = materials[0].replace(',', '|')
            polygons=redirection_root_lxml.xpath("//span[text()='Polygons: ']/following-sibling::p/text()") #caution!,this must be the number of polygons
            vertex=redirection_root_lxml.xpath("//span[text()='Vertices: ']/following-sibling::p/text()")
            is_rigged=redirection_root_lxml.xpath("//span[text()='Rigged: ']/following-sibling::p/text()")
            is_animated=redirection_root_lxml.xpath("//span[text()='Animated: ']/following-sibling::p/text()")
            license=redirection_root_lxml.xpath("//span[text()='License: ']/following-sibling::p/text()")
            quality=redirection_root_lxml.xpath("//span[text()='Quality: ']/following-sibling::p/text()")
            software_for_import=redirection_root_lxml.xpath("//span[contains(text(),'Textures & Materials:')]/parent::p/preceding-sibling::p[not(child::span)]/text()") #software compatible?
            software_for_import=software_for_import[0].replace(',', '|')
            categories=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Categories:')]/parent::p/parent::div/a[not(contains(@rel,'tag'))]/text()")
            tags=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Tags: ')]/parent::p/following-sibling::a/text()")
            '''model_info={'path':'n','file_size':file_size[0],'id':id[0],'name':name,'description':description,'uri':link,
                                'browse_file_formats':"|".join(browse_file_formats),'materials':materials,'polygons':polygons[0],'vertex':vertex[0],'is_rigged':is_rigged[0],
                                'is_animated':is_animated[0],'license':license[0],'quality':quality[0],'software_for_import':software_for_import,'categories':'|'.join(categories),'tags':'|'.join(tags)}
            print(model_info)
            models_info.append(model_info)'''
            newline = 'n' + ',' + file_size[0] + ',' + id[0].replace(' ', '').replace('#', '') + ',' + name + ',' + description + ',' + link + ',' + "|".join(browse_file_formats) + ',' + materials + ',' + polygons[0] + ',' + vertex[0] + ',' + is_rigged[0] + ',' + is_animated[0] + ',' + license[0] + ',' + quality[0] + ',' + software_for_import + ',' + "|".join(categories) + ',' + "|".join(tags)
            m += 1
            print(m)
            csv_writer.writerow(newline.split(','))
        url=page.next_page_url
        time.sleep(random.uniform(1, 3))
        i=i+1

        

new loop started
max 42
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
